In [1]:
%%html
<!–– This cell should be hidden, but probably won't be on GitHub ––>
<style>.edit_mode div.cell.selected {width: 72rem !important;}</style>
<script>$("div.input:first").hide();</script>

# Advent of Code 2018
http://adventofcode.com/2018

## Day 1
Puzzle input: `input01.txt`
### Part 1

In [1]:
with open('input01.txt') as f:
    freq_change = [int(n) for n in f]
    
print(sum(freq_change))

576


### Part 2

In [2]:
last_freq = 0
freq_history = set()
n = len(freq_change)

i = 0
while last_freq not in freq_history:
    freq_history.add(last_freq)
    last_freq += freq_change[i]    
    i = (i+1) % n
    
print(last_freq)

77674


## Day 2
Puzzle input: `input02.txt`
### Part 1

In [3]:
with open('input02.txt') as f:
    words = [w.strip() for w in f]

In [4]:
from collections import Counter

twos, threes = 0, 0
for w in words:
    c = Counter(w)
    if 2 in c.values(): twos += 1
    if 3 in c.values(): threes += 1
        
print(twos * threes)

5904


### Part 2

In [5]:
from itertools import combinations

def diff_count(w1, w2):
    return sum(a != b for a,b in zip(w1, w2))

def common(w1, w2):
    return ''.join(a for a,b in zip(w1, w2) if a == b)

for w1, w2 in combinations(words, 2):
    if diff_count(w1, w2) == 1: print(common(w1, w2))

jiwamotgsfrudclzbyzkhlrvp


## Day 3
Puzzle input: `input03.txt`
### Part 1

In [6]:
with open('input03.txt') as f:
    claim_raw = [c.strip() for c in f]

In [7]:
from collections import defaultdict
from re import search

claim_regex = r'\#(\d+)\s+\@\s+(\d+),(\d+)\:\s+(\d+)x(\d+)'

claim_id = defaultdict(set)
for c_raw in claim_raw:
    c_id, i, j, m, n = (int(g) for g in search(claim_regex, c_raw).groups())
    for p in range(i, i+m):
        for q in range(j, j+n):
            claim_id[(p,q)].add(c_id)
            
print(sum(len(claims) > 1 for claims in claim_id.values()))

104241


### Part 2

In [8]:
claim_set = set.union(*claim_id.values())

for position, claims in claim_id.items():
    if len(claims) > 1:
        claim_set -= claim_id[position]
                
print(claim_set)

{806}


## Day 4
Puzzle input: `input04.txt`
### Part 1

In [9]:
with open('input04.txt') as f:
    guard_records = [g.strip() for g in f]

In [10]:
from datetime import datetime, timedelta

guard_regex1 = '\[.+\] Guard \#(\d+)'
guard_regex2 = '\[(\d+)\-(\d+)\-(\d+)\ (\d+)\:(\d+)\] (.+) .+'

asleep = defaultdict(int)
for gr in sorted(guard_records):
    try:
        curr_guard = int(search(guard_regex1, gr).groups()[-1])
    except:
        groups = search(guard_regex2, gr).groups()
        yr, mo, da, hr, mi = (int(g) for g in groups[:-1])
        dt2 = datetime(yr, mo, da, hr, mi)
        if 'falls' in groups[-1]: 
            dt1 = dt2
        if 'wakes' in groups[-1]:
            dt = dt1
            while dt < dt2:
                asleep[(curr_guard, dt.minute)] += 1
                dt += timedelta(minutes = 1)
            dt1 = dt2
            
best_guard = sorted([
    (sum(v for k,v in asleep.items() if k[0] == g), g)
    for g in set(k[0] for k in asleep.keys())
])[-1][1]

best_minute = sorted([
    (v,k) for k,v in asleep.items() 
    if k[0] == best_guard
])[-1][1]

print(best_minute[0] * best_minute[1])

3212


### Part 2

In [11]:
best_minute = sorted([(v,k) for k,v in asleep.items()])[-1][1]
print(best_minute[0] * best_minute[1])

4966


## Day 4: Alternative solution with `pandas`
Puzzle input: `input04.txt`
### Part 1

In [12]:
import pandas as pd

df = (
    pd.read_csv('input04.txt', header = None)
    .sort_values(0).iloc[:,0].str
    .extract(r'\[(\d+)\-(\d+)\-(\d+)\ (\d+)\:(\d+)\] (.+)')
)

df['guard_id'] = (
    df[5].str.extract(r'Guard \#(\d+)', expand = False)
    .fillna(method = 'ffill').astype(int)
)

df['date'] = df.iloc[:,:5].apply(
    lambda row: datetime(*[int(r) for r in row]), 
    axis = 1
)

df['asleep'] = df[5].str.contains('sleep')

date_min, date_max = df.date.min(), df.date.max()
total_minutes = int((date_max - date_min).total_seconds()) // 60

df = pd.DataFrame({
    'date' : [date_min + timedelta(minutes = m) 
              for m in range(total_minutes + 1)]
}).merge(df, on = 'date', how = 'left')[['guard_id', 'date', 'asleep']]

df['minute'] = df.date.apply(lambda dt: dt.minute)

df = (
    df.drop('date', 1).fillna(method = 'ffill')
    .groupby(['guard_id', 'minute'], as_index = False).sum()
)

best_minute = (
    df[df.guard_id == df.groupby('guard_id').sum().asleep.idxmax()]
    .sort_values('asleep')
).iloc[-1]

print(int(best_minute.guard_id * best_minute.minute))

3212


### Part 2

In [13]:
best_minute = df.sort_values('asleep').iloc[-1]
print(int(best_minute.guard_id * best_minute.minute))

4966
